In [1]:
import serial
import time
import torch
from datetime import datetime
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

import cv2
import numpy as np
import matplotlib.pyplot as plt

from models.models import BasicLSTM
from utils.datasets import EMGO_dataset

In [2]:
def send_data(ser, pair):
    
    ser.flushInput()
    data = bytes(','.join(str(i) for i in pair) + '\n', 'utf-8')
    ser.write(data)
    
def extract_value(values, upper_leg, lower_leg):
    values[:2] *= upper_leg[0]
    values[2:] *= lower_leg[0]
    
    # Safety Measures
    values[:2] = np.clip(values[:2], 0, upper_leg[1])
    values[2:] = np.clip(values[2:], 0, lower_leg[1])
    
    return [int(v) for v in values]

In [3]:
def color_map(x):
    r = max(-np.e**(4*x-4)+1, 0)
    g = min(np.e**(8*x-4), 1)
    b = 0
    
    return (int(b*255),int(g*255),int(r*255))

def draw_leg(leg_img, circles, values):
    for c, v in zip(circles, values):
        
        d = upper_leg[1] if v in values[:2] else lower_leg[1]
        color = color_map(v/d)
        
        leg_img = cv2.circle(leg_img, c, 30, color, -1)
        leg_img = cv2.circle(leg_img, c, 33, BLACK, 5)
        
    return leg_img

In [4]:
#Leg Cords

ham_l_circle = (65, 200)
quad_l_circle = (220, 200)

anti_l_circle = (50, 500)
calve_l_circle = (185, 500)

ham_r_circle = (320, 200)
quad_r_circle = (490, 200)

anti_r_circle = (320, 500)
calve_r_circle = (455, 500)

circles_left = [quad_l_circle, ham_l_circle, calve_l_circle, anti_l_circle]
circles_right = [quad_r_circle, ham_r_circle, calve_r_circle, anti_r_circle]

BLACK = (0,0,0)

# ser = serial.Serial('COM6', 9600)

In [5]:
csv_path = "emg.csv"
dataset = EMGO_dataset(csv_path, lag=24, n_ahead=12)


input_size = 4
hidden_size = 128
num_layers = 5
output_size = dataset.n_ahead

model = BasicLSTM(input_size, hidden_size, num_layers, output_size)
model.load_state_dict(torch.load('models/pre-trained/EMG_Last.pt'))

,EMG PATH,FRAMES
0,dataset/2.0mph.csv,1800
1,dataset/2.5mph.csv,1800
2,dataset/3.1mph.csv,1800
3,dataset/Quiet_Stance_0.csv,1800
4,dataset/StandStop_2mph.csv,1800


<All keys matched successfully>

In [7]:
leg_img = cv2.imread("figures/legs.png")
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

upper_leg = [120, 65]
lower_leg = [160, 120]
FPS = 30

legs_out = cv2.VideoWriter(f"legs.mp4", fourcc, FPS, (leg_img.shape[1], leg_img.shape[0]))

for i in range(dataset.__len__()):
    
    initialFrameTime = datetime.now()
    
    emg_accel, right_emg = dataset.__getitem__(i)
    right_emg = [i.item() for i in right_emg[0, :]]
    left_emg = [i.item() for i in emg_accel[-1, :]]
    
    with torch.no_grad():
        pred_emg = [i.item() for i in model(emg_accel.view(1, 24, 4))[0, 0, :] ]
    
    
    pred_emg = extract_value(np.array(pred_emg), upper_leg, lower_leg)
    right_emg = extract_value(np.array(right_emg), upper_leg, lower_leg)
    left_emg = extract_value(np.array(left_emg), upper_leg, lower_leg)
    print(pred_emg)
    
    # send_data(ser, pred_emg)
    
    leg_img = draw_leg(leg_img, circles_left, left_emg)
    leg_img = draw_leg(leg_img, circles_right, right_emg)
    
    legs_out.write(leg_img)
    
    cv2.imshow("leg", leg_img)
    cv2.waitKey(1)
    
    
    lateFrameDifference = float( str(datetime.now() - initialFrameTime)[7:] )
    
    while (lateFrameDifference < (1/FPS)):
        lateFrameDifference = float( str(datetime.now() - initialFrameTime)[7:] )
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    # sds = input()
    
legs_out.release()
# ser.close()

[14, 24, 91, 120]
[29, 15, 63, 93]
[46, 19, 63, 47]
[50, 21, 75, 65]
[27, 17, 79, 77]
[34, 23, 38, 67]
[27, 39, 68, 83]
[45, 27, 83, 90]
[31, 44, 50, 47]
[49, 49, 101, 96]
[36, 14, 69, 67]
[30, 40, 65, 101]
[34, 36, 64, 102]
[29, 27, 103, 99]
[42, 13, 95, 62]
[32, 65, 70, 114]
[35, 39, 80, 83]
[22, 44, 56, 120]
[46, 33, 49, 111]
[40, 20, 37, 72]
[43, 30, 43, 109]
[62, 40, 67, 97]
[39, 26, 69, 48]
[35, 37, 116, 73]
[25, 38, 87, 89]
[6, 23, 77, 98]
[40, 32, 76, 60]
[34, 11, 69, 61]
[34, 45, 99, 87]
[21, 52, 88, 47]
[33, 51, 52, 85]
[42, 35, 48, 69]
[29, 58, 78, 26]
[64, 49, 81, 82]
[11, 49, 120, 66]
[34, 42, 59, 100]
[27, 15, 56, 54]
[22, 34, 34, 89]
[48, 33, 43, 57]
[48, 52, 99, 86]
[21, 21, 50, 42]
[39, 41, 97, 69]
[57, 47, 44, 99]
[27, 58, 57, 89]
[33, 36, 59, 92]
[44, 42, 67, 80]
[23, 27, 39, 112]
[31, 29, 94, 82]
[40, 31, 58, 96]
[65, 30, 55, 120]
[53, 27, 59, 106]
[33, 5, 74, 88]
[28, 39, 60, 110]
[49, 47, 40, 63]
[53, 30, 71, 103]
[22, 39, 51, 95]
[28, 37, 85, 112]
[47, 14, 111, 4

In [7]:
send_data(ser, [0,0,0,0])

In [8]:
ser.close()

In [78]:
ser = serial.Serial('COM4', 9600)  # Replace COM4 with the port name of your Arduino
c = 0

while True:
    
    
    
    
    value = 30  # Replace 42 with the integer you want to send
    ser.flushInput()  # Clear the input buffer
    ser.write(value.to_bytes(2, 'big'))  # Convert the integer to bytes and send it
    
    time.sleep(2)  # Wait for 1 second before sending the next value
    
    
    value = 0  # Replace 42 with the integer you want to send
    ser.flushInput()  # Clear the input buffer
    ser.write(value.to_bytes(2, 'big'))  # Convert the integer to bytes and send it
    
    time.sleep(2)
    
    c+=1
    

KeyboardInterrupt: 

In [10]:


emg_path = "M:/Datasets/shock_walk/processed/2.0mph.csv"
emg_data = pd.read_csv(emg_path)
display(emg_data)
# emg_data = np.transpose(emg_data, (1, 0))

# Input = emg_data[:, 0 : 12]
# Input


,L_Quad: EMG 1,L_Quad: Acc 1.X,L_Quad: Acc 1.Y,L_Quad: Acc 1.Z,L_Hamstrings: EMG 2,L_Hamstrings: Acc 2.X,L_Hamstrings: Acc 2.Y,L_Hamstrings: Acc 2.Z,L_AnteriorTibialis: EMG 3,L_AnteriorTibialis: Acc 3.X,...,R_Hamstrings: Acc 6.Y,R_Hamstrings: Acc 6.Z,R_AnteriorTibialis: EMG 7,R_AnteriorTibialis: Acc 7.X,R_AnteriorTibialis: Acc 7.Y,R_AnteriorTibialis: Acc 7.Z,R_Calf: EMG 8,R_Calf: Acc 8.X,R_Calf: Acc 8.Y,R_Calf: Acc 8.Z
0,0.070971,0.119164,0.012924,0.307524,0.002241,0.353142,0.002440,0.344716,0.006304,0.197834,...,0.012567,0.375806,0.143050,0.169350,0.013233,0.320502,0.289607,0.072672,0.432930,0.001787
1,0.070971,0.119164,0.012924,0.307524,0.002241,0.353142,0.002440,0.344716,0.006304,0.197834,...,0.012567,0.375806,0.143050,0.169350,0.013233,0.320502,0.289607,0.072672,0.432930,0.001787
2,0.070971,0.119164,0.012924,0.307524,0.002241,0.353142,0.002440,0.344716,0.006304,0.197834,...,0.012567,0.375806,0.143050,0.169350,0.013233,0.320502,0.289607,0.072672,0.432930,0.001787
3,0.025604,0.520571,0.041173,0.570102,0.543523,0.648173,0.085791,0.201149,0.267603,0.326624,...,0.012567,0.276095,0.141509,0.296914,0.031059,0.645408,0.204101,0.399392,0.254531,0.390678
4,0.177594,0.834173,0.151240,0.616488,0.358831,0.921115,0.127835,0.198388,0.225178,0.462923,...,0.055262,0.122151,0.076624,0.396534,0.035014,0.947223,0.176568,0.875469,0.082299,0.735694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,0.015741,0.658561,0.074040,0.235573,0.042215,0.265317,0.018327,0.275753,0.061783,0.477940,...,0.219943,0.182958,0.153354,0.759686,0.157974,0.238600,0.138171,0.871455,0.247629,0.757838
1796,0.020910,0.700374,0.077257,0.264929,0.103298,0.282671,0.024682,0.269563,0.063414,0.509851,...,0.165596,0.256912,0.324430,0.820816,0.082588,0.258391,0.134136,0.633790,0.108874,0.700003
1797,0.022084,0.848808,0.025790,0.253513,0.120972,0.252695,0.006355,0.254090,0.032635,0.442274,...,0.172000,0.156663,0.243400,0.761433,0.081753,0.220458,0.132893,0.583615,0.106526,0.536676
1798,0.019265,0.679468,0.012166,0.266560,0.127044,0.276361,0.015150,0.241712,0.048729,0.442274,...,0.244341,0.090926,0.147414,0.663626,0.170407,0.230353,0.255423,0.746181,0.113574,0.612460
